My Dad wants to retire soon. Like many baby boomers, his retirement will consist of several 

This is a good idea for an app

In [1]:
retire_age = 62
# TODO: Make social security vary based on the age

# Define expected monthly fash streams
e_social_security = 1800 # This might go down by $500 in the future
e_oil = 2000

Note that there is a guranteed 7% return on social security so it'll probably be worth waiting to collect on that one. http://www.socialsecuritychoices.com/info/social_security_as_investment.php. Waiting until 64 actually yields a 6% guranteed ROI assuming average life expectancy. Dad's median life expectancy is 82 years

In [2]:
# Define expected capital amounts
dc_benefit = 445e3
pension = 420e3
total = pension + dc_benefit

In [3]:
total

865000.0

My dad will want to split it between lending club and the stock market.

# The Expected Returns

In [4]:
lc_e_return = .10
stock_e_return = .07

lc_share = .7
stock_share = 1 - lc_share

In [5]:
lc_amount = lc_share * total
stock_amount = stock_share * total
print(lc_amount, stock_amount)

605500.0 259500.00000000003


In [6]:
expected_return = lc_e_return * lc_amount + stock_amount * stock_e_return

In [7]:
total_cash_flow = expected_return / 12 + e_oil + e_social_security

In [8]:
total_cash_flow

10359.583333333332

In [9]:
tax_rate = 22 / 120

In [10]:
total_cash_flow - tax_rate * total_cash_flow

8460.326388888889

# Monte Carlo

In [105]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [58]:
def stock_returns():
    return np.random.lognormal(.064, .179, 1)

In [144]:
def royalty_returns():
    # Royalties can;t go below 0
    return max(np.random.normal(2000,1500), 0)

In [125]:
def lc_returns():
    if np.random.uniform() > 0.99:
        # Lending club goes bankrupt with probability .01
        if np.random.uniform() > 0.5:
            # Get half back
            lc_returns = 0.5
        else:
            # Judge Eliminates everything
            lc_returns = 0
    else:
        lc_returns = np.random.normal(1.11,.02)
    return lc_returns

Note that this is for any given year

In [127]:
simulations = []
for simulation in range(100000):
    lc_cash = (lc_returns() - 1) * lc_amount
    stock_cash = (stock_returns() - 1) * stock_amount
    royalty_cash = royalty_returns() * 12
    
    simulations.append(royalty_cash + lc_cash + stock_cash + e_social_security)

In [191]:
def sim_year(total):
    simulations = []
    for simulation in range(10000):
        lc_cash = (lc_returns()) * lc_amount
        stock_cash = (stock_returns()) * stock_amount
        royalty_cash = royalty_returns() * 12

#         simulations.append(royalty_cash + lc_cash + stock_cash + e_social_security - total)
        simulations.append(royalty_cash + lc_cash + stock_cash + e_social_security)
    return simulations

## Monte Carlo Simulations of Many years

### Ways to Approach this

1. List of lists. 
    - Retirment simulations Length N by Y
2. Do worst 10% reliabily

In [176]:
1e5 * 1e5

10000000000.0

In [171]:
new_total = sim[0] if sim[0] < total else total

The problem is, once we are using this Monte Carlo tecnique, and then just taking the second, percentile, we are basically 

In [247]:
# Numbers aren;t updating after the first year

def sim_retirement(total, years=30):
    retirment_simulations = []
    for i in range(10000)[:1]:
        yearly_result = [total]
        for year in range(years):
            simulations = sim_year(total)
            worst_10 = np.percentile(simulations, 60)
            min_withdrwawal = 4000 * 12
            total = worst_10 - min_withdrwawal
            yearly_result.append(total)
            print(total, worst_10)
        print(yearly_result)
        

In [248]:
sim_retirement(total)

941416.533298 989416.533298
940200.036404 988200.036404
941308.730208 989308.730208
940301.741112 988301.741112
941657.317889 989657.317889
940767.214263 988767.214263
940836.657109 988836.657109
940426.01246 988426.01246
940967.554872 988967.554872
941701.170968 989701.170968
940639.423024 988639.423024
940197.735686 988197.735686
940918.987213 988918.987213
941504.867191 989504.867191
941726.434888 989726.434888
941624.731364 989624.731364
941835.083709 989835.083709
941118.012655 989118.012655
940911.589422 988911.589422
942068.875339 990068.875339
941583.368516 989583.368516
942138.787882 990138.787882
941949.9883 989949.9883
942356.841981 990356.841981
941577.551085 989577.551085
941281.123602 989281.123602
941540.728512 989540.728512
941292.168412 989292.168412
939669.980759 987669.980759
940562.031354 988562.031354
[865000.0, 941416.53329750791, 940200.03640428837, 941308.73020757316, 940301.74111187854, 941657.31788870681, 940767.21426270495, 940836.6571093899, 940426.012460427

In [229]:
arr = np.array(simulations) / 12 # Divide by 12 to get monthly amounts

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

> <ipython-input-229-096029c62fcd>(1)<module>()
----> 1 arr = np.array(simulations) / 12 # Divide by 12 to get monthly amounts

ipdb> q


In [206]:
arr = np.array(simulations)

In [214]:
# Monthly Amounts 
np.percentile(arr, 2), np.percentile(arr, 50), np.percentile(arr, 95)

(866451.73016278504, 974613.50463586184, 1074963.0538279477)

In [145]:
sim = [royalty_returns() for _ in range(1000)]

In [146]:
sim = np.array(sim)

In [147]:
sim.std()

1368.6875467521743

In [151]:
np.percentile(sim, 8)

0.0

# Monte Carlo Over Time

The next step is going to be to simulate the results over time. This is difficult because of multicollinearity. 

In any case, this amount of money when dedicated purely to cash flow is 850k

# Ways to Approach the Monte Carlo Simulation

- Fit lending club returns according to my own distribution $ N \sim (.11, .1 $)
- Fit oil returns according to $max ( N \sim (2000, 1000), 0 )$
- Correlate them with the stock market (oil returns .2, lending club .1)

- https://www.portfoliovisualizer.com/monte-carlo-simulation#analysisResults
-https://www.quora.com/What-is-the-best-family-of-distributions-to-model-stock-market-returns
- http://stackoverflow.com/questions/6620471/fitting-empirical-distribution-to-theoretical-ones-with-scipy-python For fitting the S&P Returns
- http://www.prospercuity.com/modified-distributions.htm Log normal with mean 6.4% and standard deviation 17.9%. Also have the data there